# Financial Analysis with TA-Lib and PyNance

This notebook demonstrates how to perform technical analysis on stock data using TA-Lib and PyNance.

In [ ]:
# Install required packages if not already installed
!pip install yfinance pandas numpy matplotlib seaborn ta-lib pynance

In [ ]:
# Import required libraries
import sys
import os
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

# Add scripts directory to path
script_dir = os.path.abspath(os.path.join('..', 'scripts'))
if script_dir not in sys.path:
    sys.path.append(script_dir)
    print(f\"Added {script_dir} to Python path\")
else:
    print(\"Script directory already in path\")

In [ ]:
from financial_analysis import analyze_stock

# Example: Analyze Apple stock (AAPL) for the last year
symbol = 'AAPL'
df_analysis = analyze_stock(symbol)

# Display the last few rows of the analysis
if df_analysis is not None:
    display(df_analysis[['Open', 'High', 'Low', 'Close', 'Volume', 'SMA_20', 'RSI', 'MACD']].tail())

In [ ]:
# Additional Analysis: Compare multiple stocks
def compare_stocks(symbols, start_date=None, end_date=None):
    """Compare multiple stocks based on cumulative returns."""
    import yfinance as yf
    import pandas as pd
    
    # Download data for all symbols
    data = yf.download(symbols, start=start_date, end=end_date)['Adj Close']
    
    # Calculate daily returns and cumulative returns
    returns = data.pct_change().dropna()
    cumulative_returns = (1 + returns).cumprod()
    
    # Plot cumulative returns
    plt.figure(figsize=(12, 6))
    for symbol in symbols:
        plt.plot(cumulative_returns.index, cumulative_returns[symbol], label=symbol)
    
    plt.title('Cumulative Returns Comparison')
    plt.xlabel('Date')
    plt.ylabel('Cumulative Returns')
    plt.legend()
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    
    # Save the plot
    os.makedirs('../output', exist_ok=True)
    plt.savefig(f'../output/compare_{"_".join(symbols)}.png')
    plt.show()
    
    return returns, cumulative_returns

# Example: Compare tech stocks
tech_stocks = ['AAPL', 'MSFT', 'GOOGL', 'AMZN', 'META']
returns, cumulative_returns = compare_stocks(tech_stocks)

# Display correlation matrix
plt.figure(figsize=(10, 8))
sns.heatmap(returns.corr(), annot=True, cmap='coolwarm', center=0)
plt.title('Correlation Matrix of Daily Returns')
plt.tight_layout()
plt.savefig('../output/correlation_matrix.png')
plt.show()

In [ ]:
# Risk Analysis
def calculate_risk_metrics(returns):
    """Calculate risk metrics for the given returns."""
    # Annualized return
    annual_return = returns.mean() * 252
    
    # Annualized volatility
    annual_volatility = returns.std() * np.sqrt(252)
    
    # Sharpe ratio (assuming 0% risk-free rate for simplicity)
    sharpe_ratio = annual_return / annual_volatility
    
    # Create a DataFrame with the metrics
    metrics = pd.DataFrame({
        'Annual Return': annual_return,
        'Annual Volatility': annual_volatility,
        'Sharpe Ratio': sharpe_ratio
    })
    
    return metrics

# Calculate and display risk metrics
risk_metrics = calculate_risk_metrics(returns)
print(\"Risk Metrics for Each Stock:\")
display(risk_metrics)